# Template for Exporatory Analysis with AD
This template demonstrates, how to use AD from Jupyter notebook.

_NOTE_: This file needs to be updated with demonstrative example.

## Imports
Importing general data manipulation and visualization libraries as well as some helper libraries. We also import a module ConsumerFile, which enable us to interact with the system.

In [ ]:
# imports

# general imports
import pandas as pd
import json
import plotly.express as px
import ast

# local imports
import sys
sys.path.insert(0,'../../src')
sys.path.insert(0,'../..')

from consumer import ConsumerFile

## Defining experimental parameters and data

In [ ]:
# defining data files
N = [100, 200, 1000]
X = [0.5, 1, 2, 3]

data_files = {
    "B200_hour_SS_input": ["52", "56"],
    "B300_hour_SS_input": ["51", "57"]
}
base_data_location = "../../data/JEMS/"

## Running the experiments

In [ ]:
# use a configuration
with open("../../configuration/JEMS/welford.json") as c:
    conf = json.load(c)
    # run experiments for all data files
    for data_file in data_files:
        sensors = data_files[data_file]

        # read data into a dataframe
        data_location = base_data_location + data_file + ".csv"
        df = pd.read_csv(data_location)

        # iterate over sensors
        for sensor in sensors:
            # prepare file
            vals = df[["timestamp", sensor]]
            val_file_name = "data/{}_sensor{}.csv".format(data_file, sensor)
            vals.to_csv(val_file_name, index=False)
            conf["file_name"] = val_file_name

            # loop through hyperparameters
            for n in N:
                for x in X:
                    # set configurations
                    conf["anomaly_detection_conf"][0]["N"] = n
                    conf["anomaly_detection_conf"][0]["X"] = x
                    file_name = "{}_sensor{}_welford_N{}_X{}.csv".format(data_file, sensor, n, x)
                    conf["anomaly_detection_conf"][0]["output_conf"][0]["file_name"] = file_name

                    # run anomaly detection
                    consumer = ConsumerFile(conf)
                    consumer.read()
                    print(file_name)

# Visualization of the results

In [ ]:
# check the results from log file
res = pd.read_csv("log/sensor53_welford_N1000_X2.csv", index_col="timestamp")

res["status_code"] = res["status_code"].astype(str)
res["value"] = res.value.apply(lambda x: ast.literal_eval(x)[0])

fig = px.scatter(res, y="value", color="status_code", hover_data=['value'], color_discrete_sequence=["blue", "green", "yellow", "red"])
fig.show()